In [1]:
import pandas as pd

In [2]:
#OPENAI_API_KEY="[API_KEY_HERE]"

In [7]:
p_in="/Users/vorsi/Dropbox/bot and agenda-setting/codes and data/reg_table_with_ctrls_bot_prob/"

In [ ]:
test_bots=pd.read_csv("/Users/vorsi/Dropbox/bot and agenda-setting/orsi/bot.csv")

In [ ]:
len(test_bots)

In [ ]:
test_bots

In [ ]:
def get_user_id(row):
    return int(row['url'].split("user_id=")[-1])

In [ ]:
test_bots['user_id']=test_bots.apply(get_user_id,1)

In [23]:

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", 
     "content": """Can you tell me if this Twitter account at the following link 
     is a human user or a bot: """+test_bots['url'][0]}
  ]
)

print(completion.choices[0].message.content)

As an AI language model, I do not have the ability to access external links. However, you can use a tool like Botometer (botometer.osome.iu.edu) to check if a Twitter account is likely to be a bot or a human user based on various factors.


In [ ]:
test_bots['user_id'][0]

In [33]:
# Check a single account by screen name
#result = bom.check_account('@clayadavis')

# Check a single account by id
result = bom.check_account(test_bots['user_id'][0])

# Check a sequence of accounts
#accounts = ['@clayadavis', '@onurvarol', '@jabawack']
#for screen_name, result in bom.check_accounts_in(accounts):
    # Do stuff with `screen_name` and `result`

user: Twitter user object (from the request) plus the language inferred from the majority of tweets

raw scores: bot scores in the [0,1] range, both English (using all features) and Universal (using only language-independent features); in each case we have the overall score and the sub-scores for each bot class (see below for subclass names and definitions)

display scores: same as raw scores, but in the [0,5] range

cap: Complete Automation Probability (CAP) is the conditional probability that accounts with a score equal to or greater than this are automated; based on inferred language

Bot types:

fake_follower: bots purchased to increase follower counts 

self_declared: bots from botwiki.org 

astroturf: manually labeled political bots and accounts involved in follow trains that systematically delete content

spammer: accounts labeled as spambots from several datasets

financial: bots that post using cashtags

other: miscellaneous other bots obtained from manual annotation, user feedback, etc.


In [39]:
R=[]
errrors=[]
for user in list(test_bots['user_id']):
    try:
        result = bom.check_account(user)
        R.append(result)
    except:
        errrors.append(user)

Bot scores are displayed on a 0-to-5 scale with zero being most human-like and five being the most bot-like. A score in the middle of the scale is a signal that our classifier is uncertain about the classification. On the Botometer Pro API, this is called display score. The API also provides the raw score in the range 0-1.

While bot scores are useful for visualization and behavior analysis, they don't provide enough information by themselves to classify an account. A more meaningful way to interpret a score is to ask: "What are the chances that an account with a bot score higher than this account is human, or automated?" To answer this question, the Botometer API provides the so-called CAP, defined as the probability, according to our models, that an account with this score or greater is controlled by software, i.e., is a bot. (For the statisticians, this conditional probability calculation uses Bayes' theorem to take into account an estimate of the overall prevalence of bots, so as to balance false positives with false negatives.)

For example, suppose an account has a raw bot score of 0.96/1 (equivalent to 4.8/5 display score on the website) and CAP 90%. This means that 90% of accounts with a raw bot score above 0.96 are labeled as bots, or, as indicated on the website, 10% of accounts with a bot score above 4.8/5 are labeled as humans. In other words, if you use a threshold of 0.96 on the raw bot score (or 4.8 on the display score) to classify accounts as human/bot, you would wrongly classify 10% of accounts as bots -- a false positive rate of 10%.

Therefore, if binary classification is necessary, you could set a threshold on the CAP based on a statistical test. You may want the false positive rate to be pretty small, say less than 5%. To this end, you could label accounts with CAP above 95% as bots. Depending on your application, you may want to use a less conservative threshold.

In [51]:
R[1]['display_scores']

{'english': {'astroturf': 0.0,
  'fake_follower': 1.2,
  'financial': 0.0,
  'other': 0.4,
  'overall': 0.6,
  'self_declared': 0.2,
  'spammer': 0.1},
 'universal': {'astroturf': 0.0,
  'fake_follower': 1.0,
  'financial': 0.0,
  'other': 0.7,
  'overall': 0.6,
  'self_declared': 0.0,
  'spammer': 0.2}}

In [12]:
import json
#json_object = json.dumps(R)

In [58]:
pd.DataFrame(R[1])

,cap,display_scores,raw_scores,user
english,0.502712,"{'astroturf': 0.0, 'fake_follower': 1.2, 'fina...","{'astroturf': 0.01, 'fake_follower': 0.23, 'fi...",NaN
universal,0.574269,"{'astroturf': 0.0, 'fake_follower': 1.0, 'fina...","{'astroturf': 0.0, 'fake_follower': 0.21, 'fin...",NaN
majority_lang,NaN,NaN,NaN,en
user_data,NaN,NaN,NaN,"{'id_str': '1031374994813665281', 'screen_name..."


In [59]:
#son_object

In [60]:
 
# Writing to sample.json
json_object = json.dumps(R)
with open("sampled_bots_botometer.json", "w") as outfile:
    outfile.write(json_object)

In [61]:
! ls

check_GPT.ipynb             sampled_bots_botometer.json
checlk_RF_model.ipynb


# CHECK OUR BOTS

In [9]:
# Import in bots


In [10]:
len(all_users)

122130

In [15]:
#(len(set(all_users['id']))-2000)*0.001

In [108]:
#checked_users

In [106]:
checked_users=[int(i['user']['user_data']['id_str']) for i in R]+errrors

In [19]:
import json

data = []
with open('sampled_bots_botometer_Jun23.json') as f:
    for line in f:
        data.append(json.loads(line))

In [22]:
len(data[0])

12981

In [111]:
#R=[]
#errrors=[]
count=10
for userid in list(all_users['id']):
    if userid not in checked_users and count<17250:
        checked_users.append(userid)
        count+=1
        try:
            result = bom.check_account(user)
            R.append(result)
        except:
            errrors.append(user)

In [112]:
json_object = json.dumps(R)
with open("sampled_bots_botometer.json", "w") as outfile:
    outfile.write(json_object)

In [29]:
17260-2980

14280

In [31]:
bom.wait_on_ratelimit

True

In [38]:
count=0
errrors=[]
R=[]
for userid in list(all_users['id'])[:15000]:
    count+=1
    if userid not in checked_users:
        if len(R)%1000==0:
            print(len(R))
        if count%15000==0:
            print(count)
            json_object = json.dumps(R)
            with open("sampled_bots_botometer_"+str(count)+".json", "w") as outfile:
                outfile.write(json_object)
            R=[]
        try:
            result = bom.check_account(userid)
            R.append(result)
        except:
            errrors.append(userid)
        checked_users.append(userid)

0
0
1000
2000
3000
4000
4000
5000
5000
6000
7000
8000
15000


In [3]:
import os
import json
import pandas as pd

In [4]:
p_in="/Users/vorsi/Dropbox/bot and agenda-setting/codes and data/reg_table_with_ctrls_bot_prob/"
all_users=pd.read_csv(p_in+"missing_users_botometer.csv")

In [5]:
import botometer

rapidapi_key = "3028a43109msh5fa8623a499002dp14b73cjsnd68607e9d83f"
twitter_app_auth = {
    'consumer_key': '1w9kQ5nBaPbYLb7euUsCprAhU',
    'consumer_secret': 'Toa2Zq9girGmtPpnNbYegn9k4uYEeoV3yMf5LNFLobyh2bpVc9',
    'access_token': '443619917-XLzauvkNiYwMhZcug3bbQ2RGXTvyqvQ2JbYq6qxz',
    'access_token_secret': 'TTqGTLFhWpY8ItCx0hJr2ZEdd8qypMfmmkbFgYKmVMcVX',
  }
bom = botometer.Botometer(wait_on_ratelimit=True,
                          rapidapi_key=rapidapi_key,
                          **twitter_app_auth)


In [ ]:
import json
count=0
errrors=[]
R=[]
for userid in list(all_users['0']):
    count+=1
    if len(R)%1000==0:
        print(len(R))
    if count%5000==0:
        print(count)
        json_object = json.dumps(R)
        with open("sampled_bots_botometer_"+str(count)+".json", "w") as outfile:
            outfile.write(json_object)
            R=[]
    try:
        result = bom.check_account(userid)
        R.append(result)
    except:
        errrors.append(userid)
json_object = json.dumps(R)
with open("sampled_bots_botometer_"+str(count)+".json", "w") as outfile:
    outfile.write(json_object)

0
1000
2000
3000
4000
50000
1000
2000
3000
4000
55000
1000
2000
3000


In [21]:
import json
json_object = json.dumps(R)
with open("sampled_bots_botometer_"+str(count)+".json", "w") as outfile:
    outfile.write(json_object)

In [14]:
errrors

[296407842,
 1379111382,
 823654111,
 982219863014821888,
 41728073,
 1696472515,
 16729309,
 826463246320275456,
 94130700,
 901223264818737153,
 414082353,
 482557536,
 882711275582758912,
 2775466447,
 73924989,
 321750373,
 297294630,
 578367131,
 732652693,
 1195394580671160320,
 2902773399,
 1040852552,
 573499245,
 81138624,
 1192832250305339392,
 757119699763159041,
 1578802808,
 920909274951196672,
 1615355046,
 996768294773448704,
 3164876977,
 17377638,
 2370446516,
 1915413596,
 2399391672,
 478295779,
 1106890509040185344,
 832270149805764612,
 881246519306878976,
 4839562197,
 19130934,
 3740988140,
 29116832,
 1045776174,
 741147654319656961,
 2862617399,
 1099632382418800640,
 161988348,
 39469444,
 228013171,
 700652670177021952,
 70813486,
 1510708483,
 613801142,
 1041162174222163968,
 3036925536,
 440474833,
 3217076560,
 742295535051264000,
 54259481,
 1089064701101694976,
 187425654,
 558466663,
 1170605568647663616,
 246357481,
 477714810,
 501092912,
 37054972,
